### Min Cost Objective

$$ \min \sum_{t=1}^{7}Labor\_Cost\_Doctor * D_t + Labor\_Cost\_Nurse * N_t + C\_N\_Mask * N\_Mask_t + C\_S\_Mask * S\_Mask_t + C\_Vent * Vent_t + C\_Glove * Glove_t + C\_Gown * Gown_t + C\_Eye\_Pro * Eye\_Pro_t + C\_Flu * Flu_t + C\_Staff\_Gown * Staff\_Gown_t$$
   s.t.  
   $$ 4\_D\_Req * D_t + 4\_N\_Req * N_t \ge 4\_Pat_t$$
   $$ 3\_D\_Req * D_t + 3\_N\_Req * N_t \ge 3\_Pat_t$$
   $$ 2\_D\_Req * D_t + 2\_N\_Req * N_t \ge 2\_Pat_t$$
   $$ 1\_N\_Req * N_t \ge 1\_Pat_t $$
   $$ D_t = D_{t-1} + On\_D_t - Off\_D_t $$
   $$ N_t = N_{t-1} + On\_N_t - Off\_N_t $$
   $$Adm\_Pat_t = 4\_Pat_t + 3\_Pat_t + 2\_Pat_t $$
   $$Staff_t = D_t + N_t $$
   $$Tot\_Pat_t = 4\_Pat_t + 3\_Pat_t + 2\_Pat_t + 1\_Pat_t $$
   $$ Adm\_Pat_t \le Num\_Beds $$
   $$ Gown_{t-1} + Gown\_Order_t = Adm\_Pat_t + Gown_t $$
   $$ Eye\_Pro_{t-1} + Eye\_Pro\_Order_t = Staff_t + Eye\_Pro_t $$
   $$ N\_Mask_{t-1} + N\_Mask\_Order_t = Staff_t + N\_Mask_t $$
   $$ Staff\_Gown_{t-1} + Staff\_Gown\_Order_t = Staff_t + Staff\_Gown_t $$
   $$ S\_Mask_{t-1} + S\_Mask\_Order_t = Tot\_Pat_t + S\_Mask_t $$
   $$ Flu_{t-1} + Flu\_Order_t = Req\_Flu * Adm\_Pat_t + Flu_t $$
   $$ Glove_{t-1} + Glove\_Order_t = Req\_Glove * Staff_t + Glove_t$$
   $$ Vent_{t-1} + Vent\_Order_t = 4\_Pat_t + 3\_Pat_t + Vent_t $$
   $$ Gown_0 = Int\_Gown $$
   $$ Eye\_Pro_0 = Int\_Eye\_Pro $$
   $$ N\_Mask_0 = Int\_N\_Mask $$
   $$ Staff\_Gown_0 = Int\_Staff\_Gown $$
   $$ S\_Mask_0 = Int\_S\_Mask $$
   $$ Flu_0 = Int\_Flu $$
   $$ Glove_0 = Int\_Glove $$
   $$ Gown_t, Vent_t, Eye\_Pro_t, N\_Mask_t, Staff\_Gown_t, S\_Mask_t, Flu_t, Glove_t, D_t, N_t, Gown\_Order_t, Eye\_Pro\_Order_t, N\_Mask\_Order_t, Staff\_Gown\_Order_t, S\_Mask\_Order_t, Flu\_Order_t, Glove\_Order_t, Vent\_Order_t \ge 0 $$

### Patient Treatment Objective

* Create Treatment Path  

* Include time for each step and capacities for each step  

* Create logical constraints for different patient severities


In [ ]:
using JuMP, Clp

m = Model(Clp.Optimizer)

# gowns
@variable(m, gown[days] >= 0)
@variable(m, gown_order[days] >= 0)
@variable(m, staff_gown[days] >= 0)
@variable(m, staff_gown_order[days] >= 0)

# gloves
@variable(m, glove[days] >= 0)
@variable(m, glove_order[days] >= 0)

# masks
@variable(m, n95_mask[days] >= 0)
@variable(m, n95_mask_order[days] >= 0)
@variable(m, surg_mask[days] >= 0)
@variable(m, surg_mask_order[days] >= 0)

# eye protection
@variable(m, shield[days] >= 0)
@variable(m, shield_order[days] >= 0)

# ventilators
@variable(m, vent[days] >= 0)
@variable(m, vent_order[days] >= 0)

# flu patients
@variable(m, fluid[days] >= 0)
@variable(m, fluid_order[days] >= 0)

# doctors and nurses on shift
@variable(m, n[days] >= 0)
@variable(m, d[days] >= 0)

# doctors and nurses added to shift
@variable(m, n_on[days] >= 0)
@variable(m, d_on[days] >= 0)

# doctors and nurses taken off of shift
@variable(m, n_off[days] >= 0)
@variable(m, d_off[days] >= 0)

# initial supplies
gown[0] == 1320)
staff_gown[0] == 1320)
glove[0] == 8250)
n95_mask[0] == 186)
surg_mask[0] == 330)
shield[0] == 69)
vent[0] == 16)
fluid[0] == 71904)
d[0] == 80)
n[0] == 160)

# constants
num_beds == 102) # total in hospital
req_staff_gown == 440) # per staff per day

# supplies needed per patient per day
req_gown == 440)
req_glove == 2750)
req_n95_mask == 62)
req_surg_mask == 110)
req_fluid == 23968)

# staff needed per patient per day depending on patient severity
1_n_req == 1/4)
2_n_req == 1/4)
3_n_req == 1/2)
4_n_req == 1/2)
2_d_req == 1/20)
3_d_req == 1/14)
4_d_req == 1/14)

# patients
@constraint(m, tot_pat[days] == pat_1[days] + pat_2[days] + pat_3[days] + pat_4[days])
@constraint(m, adm_pat[days] == pat_2[days] + pat_3[days] + pat_4[days])
@constraint(m, adm_pat[days] <= num_beds)

# staff
@constraint(m, staff[days] == d[days] + n[days])
@constraint(m, n[days] == n[days-1] + n_on[days] - n_off[days])
@constraint(m, d[days] == d[days-1] + d_on[days] - d_off[days])
@constraint(m, pat_1[days] <= 1_n_req * n[days])
@constraint(m, pat_2[days] <= 2_n_req * n[days] + 2_d_req * d[days])
@constraint(m, pat_3[days] <= 3_n_req * n[days] + 3_d_req * d[days])
@constraint(m, pat_4[days] <= 4_n_req * n[days] + 4_d_req * d[days])

# supplies
@constraint(m, gown[days-1] + gown_order[days] == gown[days] + req_gown)
@constraint(m, staff_gown[days-1] + staff_gown_order[days] == staff_gown[days] + req_staff_gown)
@constraint(m, glove[days-1] + glove_order[days] == glove[days] + req_glove)
@constraint(m, n95_mask[days-1] + n95_mask_order[days] == n95_mask[days] + req_n95_mask)
@constraint(m, surg_mask[days-1] + surg_mask_order[days] == surg_mask[days] + req_surg_mask)
@constraint(m, shield[days-1] + shield_order[days] == shield[days] + staff[days])
@constraint(m, vent[days-1] + vent_order[days] == vent[days] + pat_4[days] + pat_3[days])
@constraint(m, fluid[days-1] + fluid_order[days] == fluid[days] + req_fluid)

@objective(m, Min, d_cost * d[days] + n_cost * n[days] + n95_mask_cost * n95_mask[days] + surg_mask_cost * surg_mask[days] + gown_cost * gown[days] + glove_cost * glove[days] + shield_cost * shield[days] + vent_cost * vent[days] + fluid_cost * fluid[days] + staff_gown_cost * staff_gown[days])
